In [132]:
import sys
sys.path.append('../../../src/')

import os
import pandas as pd
import numpy as np
import config
import utils
import torch
import pickle

from Model import Impf_DNAMLP
from torch.nn.functional import softmax

In [133]:
fold = '3.4'
outer_fold = '1.0'
group = 'Sella'

In [134]:
dat_cfg = config.data_config
clf_cfg = config.classifier_config
impf_cfg = config.impf_config
TRAIN_CSV_DIR = dat_cfg['TRAIN_CSV_DIR']
TEST_CSV_DIR = dat_cfg['TEST_CSV_DIR']
OUTER_FOLD_CSV_FILE = os.path.join(TEST_CSV_DIR, group, f'{outer_fold}_test.csv')
FOLD_CSV_FILE = os.path.join(TEST_CSV_DIR, group, f'{fold}_test.csv')
device = impf_cfg['device']
device

'cpu'

In [135]:
alg = 'RF'
    
with open(os.path.join(impf_cfg['IMPORTANT_FEATURES_DIR'], alg, group, f'{outer_fold}_combined.pkl'), 'rb') as handle:
    impf = pickle.load(handle)
print(len(impf))

2095


In [136]:
df = pd.read_csv(FOLD_CSV_FILE, index_col=0)
df_test = pd.read_csv(OUTER_FOLD_CSV_FILE, index_col=0)
df.shape, df_test.shape

((42, 10002), (64, 10002))

In [137]:
list_df = sorted(list(df.columns))
list_df_test = sorted(list(df_test.columns))
print(list_df == list_df_test)

set_df = set(list_df)
set_df_test = set(list_df_test)
print(set_df == set_df_test)

False
False


In [138]:
print(len(set_df - set_df_test))
print(len(set_df_test - set_df))

283
283


In [139]:
print(impf.issubset(list_df_test))

True


In [131]:
# df_test

In [129]:
pd.Series(pd.read_csv(OUTER_FOLD_CSV_FILE, index_col=0).iloc[:, -1]).value_counts()

Sella      34
Control    25
Name: class_group, dtype: int64

In [130]:
rand = np.random.randint(df_test.shape[0])
sample = df_test.iloc[rand,:].loc[impf]
df_test.iloc[rand, -2:], sample

C:\Users\OA\AppData\Local\Temp/ipykernel_15920/2921419439.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  sample = df_test.iloc[rand,:].loc[impf]


(label          CPH, PAP
 class_group       Sella
 Name: 9305216209_R04C01, dtype: object,
 cg23173466    0.145109
 cg26848718    0.035893
 cg15628917    0.144945
 cg01822130    0.080718
 cg00089384    0.843268
                 ...   
 cg06193597    0.046315
 cg17951138    0.180737
 cg26198107     0.53908
 cg08667128    0.457917
 cg21383487    0.140722
 Name: 9305216209_R04C01, Length: 2049, dtype: object)

In [77]:
sample_features = np.expand_dims(np.array(sample).astype(float), axis = 0)
sample_features, type(sample_features), sample_features.shape, type(sample_features[0][0])

(array([[0.97294145, 0.23980472, 0.14871795, ..., 0.20960063, 0.49070415,
         0.08169291]]),
 numpy.ndarray,
 (1, 2095),
 numpy.float64)

In [78]:
IMPF_MLP_STATE_FILE = os.path.join(impf_cfg['MLP_BEST_STATES_DIR'], alg, group, f'{fold}.pth')

in_features = impf_cfg['N_IMPF']
n_classes = impf_cfg['n_classes']
in_features = len(impf)

model = Impf_DNAMLP(in_features, n_classes)
model.load_state_dict(torch.load(IMPF_MLP_STATE_FILE, map_location = torch.device('cpu')))
model.to(device)

RuntimeError: Error(s) in loading state_dict for Impf_DNAMLP:
	size mismatch for densenet.0.weight: copying a param with shape torch.Size([2, 1870]) from checkpoint, the shape in current model is torch.Size([2, 2095]).

In [ ]:
sample_features.shape

(1, 2095)

In [ ]:
logits = softmax(model((torch.Tensor(sample_features).to(device))), dim = 1)
logits

tensor([[0.1068, 0.8932]], grad_fn=<SoftmaxBackward0>)